# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4843, uptime 0:00:34
mini_beamline                    RUNNING   pid 4844, uptime 0:00:34
random_walk                      RUNNING   pid 4845, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4846, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4847, uptime 0:00:34
simple                           RUNNING   pid 4848, uptime 0:00:34
thermo_sim                       RUNNING   pid 4849, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4850, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-21 19:56:14
Persistent Unique Scan ID: 'ef58efe0-4de1-442f-acdc-fffd96a7acd5'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+


|         1 | 19:56:14.2 |     -1.000 |      95571 |
|         2 | 19:56:14.2 |     -0.500 |      99166 |
|         3 | 19:56:14.3 |      0.000 |      99705 |
|         4 | 19:56:14.3 |      0.500 |      99440 |
|         5 | 19:56:14.3 |      1.000 |      97922 |


+-----------+------------+------------+------------+
generator scan ['ef58efe0'] (scan num: 3)





('ef58efe0-4de1-442f-acdc-fffd96a7acd5',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/ef58efe0-4de1-442f-acdc-fffd96a7acd5-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-21 19:56:14
Persistent Unique Scan ID: 'd2bf843f-3661-4e19-b36c-17a578e97c81'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 19:56:14.8 |          -1 | 1110694407 |


|         2 | 19:56:14.9 |          -0 | 1117542571 |


|         3 | 19:56:15.0 |           0 | 1126295451 |


|         4 | 19:56:15.1 |           0 | 1129185220 |


|         5 | 19:56:15.2 |           1 | 1130959224 |


+-----------+------------+-------------+------------+
generator scan ['d2bf843f'] (scan num: 4)





('d2bf843f-3661-4e19-b36c-17a578e97c81',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-21 19:56:15
Persistent Unique Scan ID: 'cd7677e8-fed4-4bde-8c98-b9339848cc32'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 19:56:15.8 | 1128019997 |


|         2 | 19:56:16.7 | 1071576534 |


|         3 | 19:56:17.7 | 1059467168 |


...

|        28 | 19:56:42.8 | 1116281301 |


|        29 | 19:56:43.8 | 1123553872 |


|        30 | 19:56:44.8 | 1068237200 |


+-----------+------------+------------+
generator count ['cd7677e8'] (scan num: 5)





('cd7677e8-fed4-4bde-8c98-b9339848cc32',)